In [1]:
import numpy as np
import pandas as pd

### Este notebook auna todo el preprocesado necesario para transformar los datos de la carpeta RAW a PREPARED:

In [2]:
path_in  = "../raw/trabajo6.csv"
path_out = "../partitioned/"
path_out_csv = "../partitioned/mini.csv"

In [3]:
df = pd.read_csv(path_in)

### One Hot Encoding

In [4]:
df = df.join(pd.get_dummies(df['type']))

### Nuevas variables

In [5]:
df['diffOrigen'] = df['newbalanceOrig'] - df['oldbalanceOrg']
df['diffDestino'] = df['newbalanceDest'] - df['oldbalanceDest']

mean_oldOrg  = df['oldbalanceOrg'].mean()
mean_oldDest = df['oldbalanceDest'].mean()

# Para evitar NaN. No afecta la distribución de los datos.
df['cambioOrigen'] = (df['diffOrigen'] + 0.01)/(df['oldbalanceOrg'] + 1e3)
df['cambioDestino'] = (df['diffDestino'] + 0.01)/(df['oldbalanceDest'] + 1e3)

### Particionamos el dataset en 12 partes:

In [6]:
df['partition'] = np.random.randint(12, size=len(df))

In [7]:
df.partition.value_counts()

11    531233
0     530928
6     530550
7     530454
9     530364
2     530328
8     529939
4     529891
1     529856
3     529830
10    529790
5     529457
Name: partition, dtype: int64

### Write Prepared

In [8]:
# Necesita paquete 'pyarrow'

In [9]:
df.to_parquet(path_out, partition_cols='partition', compression='snappy')

In [4]:
# Para leer el parquet:
p_df = pd.read_parquet(path_out).drop(columns=['partition'])

In [10]:
df[df.partition == 0].to_csv(path_out_csv)